Importazione Librerie per pre-processing

In [94]:
import pandas as pd
import re
import numpy as np


Importazione dei dataset con i tweet

In [95]:
spurs = pd.read_json('Spurs.json', lines=True)
city = pd.read_json('ManchesterCity.json', lines=True)
match = pd.read_json('Match.json', lines = True)

Concatenazione dei dataset ed eliminazione dei duplicati, si conservano solo i tweet non presenti nei dataset specifici delle due squadre

In [96]:
all_tweets = pd.concat([spurs, city, match])
len(all_tweets)

84836

In [97]:
df = all_tweets.drop_duplicates(["user_id", "username", "screen_name", "text", "created_at"], keep = False)
len(df)

61507

Check sui type del dataset

In [ ]:
df = df.reset_index().copy()
df = df.reindex().copy()
df.dtypes

index                     int64
user_id                   int64
username                 object
screen_name              object
text                     object
full_text                object
hashtags                 object
retweeted                  bool
retweet_count             int64
favorite                  int64
followers                 int64
created_at       datetime64[ns]
dtype: object

Il type degli attributi è corretto tranne per user_id, si provvede a correggerlo

In [ ]:
df["user_id"] = df["user_id"].astype(str, copy = False)
df.dtypes

index                     int64
user_id                  object
username                 object
screen_name              object
text                     object
full_text                object
hashtags                 object
retweeted                  bool
retweet_count             int64
favorite                  int64
followers                 int64
created_at       datetime64[ns]
dtype: object

Check su presenza retweet o favorite

In [ ]:
check_rt = df[df['retweeted'] == True]
len(check_rt)

0

In [ ]:
check_fv = df[df['favorite'] == 1]
len(check_fv)

0

In [ ]:
check_rtc = df[df['retweet_count'] > 0]
len(check_rtc)

0

Non sono presenti retweet o favorite

Si eliminano i campi non rilevanti e si riordinano gli attributi

In [98]:
df = df.drop(['retweet_count', 'favorite', 'retweeted'], axis = 1)
df.head(15)

,user_id,username,screen_name,text,full_text,hashtags,followers,created_at
0,69871991,James,JimmyBronze,There will be positives today whatever happens...,,[thfc],263,2021-08-15 14:54:30
1,85367934,Paul Mckenzie,MaccaTHFC,"@TheSpursgirl @thfc @ManCity Hey Jane, where a...","@TheSpursgirl @thfc @ManCity Hey Jane, where a...",[],565,2021-08-15 14:54:30
3,27957972,Rob Aycock,RobAycock,Nuno is sending a strong message by leaving bo...,Nuno is sending a strong message by leaving bo...,[],393,2021-08-15 14:54:37
4,376333378,🍌⚽️🤩 kev bananaman 🍌⚽️🤩,BananamanKev,RT @FreeBetsFooty: 🚨 GIVEAWAY 🚨\n\nIf Son Heun...,,[THFC],1943,2021-08-15 14:54:37
5,1375839513906278403,なおやん,naoyanyan57,RT @thespursweb: 🚨 NEW: Harry Kane was NOT on ...,,[THFC],287,2021-08-15 14:54:40
6,1126381076,AJS 🇮🇱,ajsCOYS,#COYS no pressure x,,[COYS],563,2021-08-15 14:54:42
8,227448013,thejondaniels,thejondaniels,#COYS Kane was never gonna be in the squad. B...,#COYS Kane was never gonna be in the squad. B...,[COYS],2308,2021-08-15 14:54:44
9,1117479778792030208,🐐 𝓶𝓻.𝓓𝓘𝓔𝓡 🍑,ESTRELLA1882s,RT @thespursweb: 🎶 I want to be in that number...,,[COYS],151,2021-08-15 14:54:45
10,1061554145172238336,もなお☃️,monao0911,RT @thespursweb: 🎶 I want to be in that number...,,[COYS],409,2021-08-15 14:54:45
11,182948486,Bounty Junior 🇯🇲🇬🇾,mrdarren_t,This!!,,[],673,2021-08-15 14:54:46


In [104]:
df = df[['created_at', 'user_id',	'username',	'screen_name',	'text',	'full_text',	'hashtags',	'followers']]
df[df["created_at"] == "NaN"]

,created_at,user_id,username,screen_name,text,full_text,hashtags,followers


Si crea una nuova colonna per il testo che consideri il full text dove presente e il text in caso contrario

In [ ]:
def text(row):
  if len(row['full_text']) > len(row['text']):
    return row['full_text']   
  else:
    return row['text'] 

In [ ]:
df['text_clean'] = df.apply(text, axis=1)
df.head(15)

,created_at,user_id,username,screen_name,text,full_text,hashtags,followers,text_clean
0,2021-08-15 14:54:30,69871991,James,JimmyBronze,There will be positives today whatever happens...,,[thfc],263,There will be positives today whatever happens...
1,2021-08-15 14:54:30,85367934,Paul Mckenzie,MaccaTHFC,"@TheSpursgirl @thfc @ManCity Hey Jane, where a...","@TheSpursgirl @thfc @ManCity Hey Jane, where a...",[],565,"@TheSpursgirl @thfc @ManCity Hey Jane, where a..."
2,2021-08-15 14:54:37,27957972,Rob Aycock,RobAycock,Nuno is sending a strong message by leaving bo...,Nuno is sending a strong message by leaving bo...,[],393,Nuno is sending a strong message by leaving bo...
3,2021-08-15 14:54:37,376333378,🍌⚽️🤩 kev bananaman 🍌⚽️🤩,BananamanKev,RT @FreeBetsFooty: 🚨 GIVEAWAY 🚨\n\nIf Son Heun...,,[THFC],1943,RT @FreeBetsFooty: 🚨 GIVEAWAY 🚨\n\nIf Son Heun...
4,2021-08-15 14:54:40,1375839513906278403,なおやん,naoyanyan57,RT @thespursweb: 🚨 NEW: Harry Kane was NOT on ...,,[THFC],287,RT @thespursweb: 🚨 NEW: Harry Kane was NOT on ...
5,2021-08-15 14:54:42,1126381076,AJS 🇮🇱,ajsCOYS,#COYS no pressure x,,[COYS],563,#COYS no pressure x
6,2021-08-15 14:54:44,227448013,thejondaniels,thejondaniels,#COYS Kane was never gonna be in the squad. B...,#COYS Kane was never gonna be in the squad. B...,[COYS],2308,#COYS Kane was never gonna be in the squad. B...
7,2021-08-15 14:54:45,1117479778792030208,🐐 𝓶𝓻.𝓓𝓘𝓔𝓡 🍑,ESTRELLA1882s,RT @thespursweb: 🎶 I want to be in that number...,,[COYS],151,RT @thespursweb: 🎶 I want to be in that number...
8,2021-08-15 14:54:45,1061554145172238336,もなお☃️,monao0911,RT @thespursweb: 🎶 I want to be in that number...,,[COYS],409,RT @thespursweb: 🎶 I want to be in that number...
9,2021-08-15 14:54:46,182948486,Bounty Junior 🇯🇲🇬🇾,mrdarren_t,This!!,,[],673,This!!


Si opera adesso sugli hashtag, si recuperano gli hashtag presenti nel testo

In [ ]:
def hashtags(row):
  compi1 = re.compile('(#[Mm]an[Cc]ity)')
  compi2 = re.compile('(#[Mm]anchester[Cc]ity[Ff]c)')
  compi3 = re.compile('(#[Mm]anchester[Cc]ity)')
  compi4 = re.compile('(#[Mm][Cc][Ff][Cc])')
  compi5 = re.compile('(#[Pp]remier[Ll]eague)')
  compi6 = re.compile('(#[Pp][Ll])')
  compi7 = re.compile('(#[Tt]ottenham)')
  compi8 = re.compile('(#[Ss]purs)')
  compi9 = re.compile('(#[Cc][Oo][Yy][Ss])')
  compi10 = re.compile('(#[Tt][Hh][Ff][Cc])')
  compi11 = re.compile('(#TOTMCI)')

  hast = []
  
  subs1 = compi1.search(row['text_clean'])
  subs2 = compi2.search(row['text_clean'])
  subs3 = compi3.search(row['text_clean'])
  subs4 = compi4.search(row['text_clean'])
  subs5 = compi5.search(row['text_clean'])
  subs6 = compi6.search(row['text_clean'])
  subs7 = compi7.search(row['text_clean'])
  subs8 = compi8.search(row['text_clean'])
  subs9 = compi9.search(row['text_clean'])
  subs10 = compi10.search(row['text_clean'])
  subs11 = compi11.search(row['text_clean'])

  if subs1:
    if subs1.group(1) not in df['hashtags']: 
      hast.append(subs1.group(1).strip('#'))
  if subs2:
    if subs2.group(1) not in df['hashtags']:
      hast.append(subs2.group(1).strip('#'))
  if subs3:
    if subs3.group(1) not in df['hashtags']:
      hast.append(subs3.group(1).strip('#'))
  if subs4:
    if subs4.group(1) not in df['hashtags']:
      hast.append(subs4.group(1).strip('#'))
  if subs5:   
    if subs5.group(1) not in df['hashtags']:
      hast.append(subs5.group(1).strip('#'))
  if subs6:
    if subs6.group(1) not in df['hashtags']:
      hast.append(subs6.group(1).strip('#'))
  if subs7:
    if subs7.group(1) not in df['hashtags']:
      hast.append(subs7.group(1).strip('#'))
  if subs8:
    if subs8.group(1) not in df['hashtags']:
      hast.append(subs8.group(1).strip('#'))
  if subs9:   
    if subs9.group(1) not in df['hashtags']:
      hast.append(subs9.group(1).strip('#'))
  if subs10:
    if subs10.group(1) not in df['hashtags']:
      hast.append(subs10.group(1).strip('#'))
  if subs11:
    if subs11.group(1) not in df['hashtags']:
      hast.append(subs11.group(1).strip('#'))           
  return hast        

In [ ]:
df['has']=df.apply(hashtags, axis=1)

Si concatenano gli hashtag ottenuti dallo streaming e dal testo ed eliminando i duplicati si hanno tutti gli hashtag presenti nel tweet

In [ ]:
df['hashtag_fin'] = df['hashtags'] + df['has']

In [ ]:
def hashtag_puliti(row):
  np.array(row['hashtag_fin'])
  fi= np.unique(row['hashtag_fin'])
  return fi

In [ ]:
df['compl_hashtags'] = df.apply(hashtag_puliti, axis=1)
df.head(15)

,created_at,user_id,username,screen_name,text,full_text,hashtags,followers,text_clean,has,hashtag_fin,compl_hashtags
0,2021-08-15 14:54:30,69871991,James,JimmyBronze,There will be positives today whatever happens...,,[thfc],263,There will be positives today whatever happens...,[thfc],"[thfc, thfc]",[thfc]
1,2021-08-15 14:54:30,85367934,Paul Mckenzie,MaccaTHFC,"@TheSpursgirl @thfc @ManCity Hey Jane, where a...","@TheSpursgirl @thfc @ManCity Hey Jane, where a...",[],565,"@TheSpursgirl @thfc @ManCity Hey Jane, where a...","[Spurs, COYS, thfc]","[Spurs, COYS, thfc]","[COYS, Spurs, thfc]"
2,2021-08-15 14:54:37,27957972,Rob Aycock,RobAycock,Nuno is sending a strong message by leaving bo...,Nuno is sending a strong message by leaving bo...,[],393,Nuno is sending a strong message by leaving bo...,"[COYS, THFC]","[COYS, THFC]","[COYS, THFC]"
3,2021-08-15 14:54:37,376333378,🍌⚽️🤩 kev bananaman 🍌⚽️🤩,BananamanKev,RT @FreeBetsFooty: 🚨 GIVEAWAY 🚨\n\nIf Son Heun...,,[THFC],1943,RT @FreeBetsFooty: 🚨 GIVEAWAY 🚨\n\nIf Son Heun...,[THFC],"[THFC, THFC]",[THFC]
4,2021-08-15 14:54:40,1375839513906278403,なおやん,naoyanyan57,RT @thespursweb: 🚨 NEW: Harry Kane was NOT on ...,,[THFC],287,RT @thespursweb: 🚨 NEW: Harry Kane was NOT on ...,[THFC],"[THFC, THFC]",[THFC]
5,2021-08-15 14:54:42,1126381076,AJS 🇮🇱,ajsCOYS,#COYS no pressure x,,[COYS],563,#COYS no pressure x,[COYS],"[COYS, COYS]",[COYS]
6,2021-08-15 14:54:44,227448013,thejondaniels,thejondaniels,#COYS Kane was never gonna be in the squad. B...,#COYS Kane was never gonna be in the squad. B...,[COYS],2308,#COYS Kane was never gonna be in the squad. B...,[COYS],"[COYS, COYS]",[COYS]
7,2021-08-15 14:54:45,1117479778792030208,🐐 𝓶𝓻.𝓓𝓘𝓔𝓡 🍑,ESTRELLA1882s,RT @thespursweb: 🎶 I want to be in that number...,,[COYS],151,RT @thespursweb: 🎶 I want to be in that number...,[COYS],"[COYS, COYS]",[COYS]
8,2021-08-15 14:54:45,1061554145172238336,もなお☃️,monao0911,RT @thespursweb: 🎶 I want to be in that number...,,[COYS],409,RT @thespursweb: 🎶 I want to be in that number...,[COYS],"[COYS, COYS]",[COYS]
9,2021-08-15 14:54:46,182948486,Bounty Junior 🇯🇲🇬🇾,mrdarren_t,This!!,,[],673,This!!,[],[],[]


Si assume nel tentativo di attribuire l'appartenenza ad una tifoseria che sia necessaria la presenza di hashtag all'elaborazione

In [ ]:
def selezione(row):
  lung = len(row['compl_hashtags'])
  if lung != 0:
    return 'Yes'
  else:
    return 'No' 

In [ ]:
df['Keep'] = df.apply(selezione, axis =1)
df_clean = df[df['Keep'] == "Yes"]
df_clean.head(15)

,created_at,user_id,username,screen_name,text,full_text,hashtags,followers,text_clean,has,hashtag_fin,compl_hashtags,Keep
0,2021-08-15 14:54:30,69871991,James,JimmyBronze,There will be positives today whatever happens...,,[thfc],263,There will be positives today whatever happens...,[thfc],"[thfc, thfc]",[thfc],Yes
1,2021-08-15 14:54:30,85367934,Paul Mckenzie,MaccaTHFC,"@TheSpursgirl @thfc @ManCity Hey Jane, where a...","@TheSpursgirl @thfc @ManCity Hey Jane, where a...",[],565,"@TheSpursgirl @thfc @ManCity Hey Jane, where a...","[Spurs, COYS, thfc]","[Spurs, COYS, thfc]","[COYS, Spurs, thfc]",Yes
2,2021-08-15 14:54:37,27957972,Rob Aycock,RobAycock,Nuno is sending a strong message by leaving bo...,Nuno is sending a strong message by leaving bo...,[],393,Nuno is sending a strong message by leaving bo...,"[COYS, THFC]","[COYS, THFC]","[COYS, THFC]",Yes
3,2021-08-15 14:54:37,376333378,🍌⚽️🤩 kev bananaman 🍌⚽️🤩,BananamanKev,RT @FreeBetsFooty: 🚨 GIVEAWAY 🚨\n\nIf Son Heun...,,[THFC],1943,RT @FreeBetsFooty: 🚨 GIVEAWAY 🚨\n\nIf Son Heun...,[THFC],"[THFC, THFC]",[THFC],Yes
4,2021-08-15 14:54:40,1375839513906278403,なおやん,naoyanyan57,RT @thespursweb: 🚨 NEW: Harry Kane was NOT on ...,,[THFC],287,RT @thespursweb: 🚨 NEW: Harry Kane was NOT on ...,[THFC],"[THFC, THFC]",[THFC],Yes
5,2021-08-15 14:54:42,1126381076,AJS 🇮🇱,ajsCOYS,#COYS no pressure x,,[COYS],563,#COYS no pressure x,[COYS],"[COYS, COYS]",[COYS],Yes
6,2021-08-15 14:54:44,227448013,thejondaniels,thejondaniels,#COYS Kane was never gonna be in the squad. B...,#COYS Kane was never gonna be in the squad. B...,[COYS],2308,#COYS Kane was never gonna be in the squad. B...,[COYS],"[COYS, COYS]",[COYS],Yes
7,2021-08-15 14:54:45,1117479778792030208,🐐 𝓶𝓻.𝓓𝓘𝓔𝓡 🍑,ESTRELLA1882s,RT @thespursweb: 🎶 I want to be in that number...,,[COYS],151,RT @thespursweb: 🎶 I want to be in that number...,[COYS],"[COYS, COYS]",[COYS],Yes
8,2021-08-15 14:54:45,1061554145172238336,もなお☃️,monao0911,RT @thespursweb: 🎶 I want to be in that number...,,[COYS],409,RT @thespursweb: 🎶 I want to be in that number...,[COYS],"[COYS, COYS]",[COYS],Yes
11,2021-08-15 14:54:49,91628625,SNWDN ❄️ COYS,Liamsnowdon,RT @thespursweb: 🎶 I want to be in that number...,,[COYS],806,RT @thespursweb: 🎶 I want to be in that number...,[COYS],"[COYS, COYS]",[COYS],Yes


Si tratta adesso la variabile tempo (created_at)

Si importa la libreria necessaria a trattare date e orari

In [ ]:
from datetime import *

Abbiamo inziato lo streaming dei tweet circa alle 16:55, si nota pertanto che nel dataset l'orario di creazione del tweet è riportato con due ore di differenza, presumibilmente dovute al fuso orario. Con la seguente funzione si compensa questo valore per riportarsi all'orario italiano.

In [ ]:
def time(loc_date_time):
    date_time = loc_date_time + timedelta(hours = 2)
    return date_time 

df_clean['time'] = df_clean['created_at'].apply(time)

df_clean

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,created_at,user_id,username,screen_name,text,full_text,hashtags,followers,text_clean,has,hashtag_fin,compl_hashtags,Keep,time
0,2021-08-15 14:54:30,69871991,James,JimmyBronze,There will be positives today whatever happens...,,[thfc],263,There will be positives today whatever happens...,[thfc],"[thfc, thfc]",[thfc],Yes,2021-08-15 16:54:30
1,2021-08-15 14:54:30,85367934,Paul Mckenzie,MaccaTHFC,"@TheSpursgirl @thfc @ManCity Hey Jane, where a...","@TheSpursgirl @thfc @ManCity Hey Jane, where a...",[],565,"@TheSpursgirl @thfc @ManCity Hey Jane, where a...","[Spurs, COYS, thfc]","[Spurs, COYS, thfc]","[COYS, Spurs, thfc]",Yes,2021-08-15 16:54:30
2,2021-08-15 14:54:37,27957972,Rob Aycock,RobAycock,Nuno is sending a strong message by leaving bo...,Nuno is sending a strong message by leaving bo...,[],393,Nuno is sending a strong message by leaving bo...,"[COYS, THFC]","[COYS, THFC]","[COYS, THFC]",Yes,2021-08-15 16:54:37
3,2021-08-15 14:54:37,376333378,🍌⚽️🤩 kev bananaman 🍌⚽️🤩,BananamanKev,RT @FreeBetsFooty: 🚨 GIVEAWAY 🚨\n\nIf Son Heun...,,[THFC],1943,RT @FreeBetsFooty: 🚨 GIVEAWAY 🚨\n\nIf Son Heun...,[THFC],"[THFC, THFC]",[THFC],Yes,2021-08-15 16:54:37
4,2021-08-15 14:54:40,1375839513906278403,なおやん,naoyanyan57,RT @thespursweb: 🚨 NEW: Harry Kane was NOT on ...,,[THFC],287,RT @thespursweb: 🚨 NEW: Harry Kane was NOT on ...,[THFC],"[THFC, THFC]",[THFC],Yes,2021-08-15 16:54:40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61501,2021-08-15 18:30:52,1282777116879331328,IMO BAD BOI 🤴,Luking_official,RT @OfficialSambasa: Sterling is just another ...,,[TOTMCI],3553,RT @OfficialSambasa: Sterling is just another ...,[TOTMCI],"[TOTMCI, TOTMCI]",[TOTMCI],Yes,2021-08-15 20:30:52
61502,2021-08-15 18:30:53,1231158539575603200,Stay hungry Stay foolish,EricJimmyLevi,"RT @premierleague: One round down, 37 to go. \...",,[PL],21147,"RT @premierleague: One round down, 37 to go. \...",[PL],"[PL, PL]",[PL],Yes,2021-08-15 20:30:53
61503,2021-08-15 18:30:53,246131818,Kwekuwofa,KwekuWofaGh,"RT @ind3r_07: With the Sikhs back at OT, it re...",,[OldTrafford],1107,"RT @ind3r_07: With the Sikhs back at OT, it re...",[],[OldTrafford],[OldTrafford],Yes,2021-08-15 20:30:53
61505,2021-08-15 18:30:54,1342103655248506884,dreamz 🍑🥵,dr_ea_mz,RT @OfficialSambasa: Sterling is just another ...,,[TOTMCI],1617,RT @OfficialSambasa: Sterling is just another ...,[TOTMCI],"[TOTMCI, TOTMCI]",[TOTMCI],Yes,2021-08-15 20:30:54


Dovendo far riferimento al minuto della partita si è ritenuto opportuno riportarsi ad una situazione di tempo discretizzato a 15 secondi.

In [ ]:
time_time = pd.concat([pd.Series(df_clean["time"].dt.hour),
                       pd.Series(df_clean["time"].dt.minute),
                       pd.Series(df_clean["time"].dt.second)],axis=1)

In [ ]:
time_time["hour"] = time_time.iloc[:,0]
time_time["minute"] = time_time.iloc[:,1]
time_time["second"] = time_time.iloc[:,2]

In [ ]:
time_time["second"] = time_time["second"]/15
time_time["second"] = time_time["second"].round()
time_time["second"] = time_time["second"]*15
time_time

,time,time,time,hour,minute,second
0,16,54,30,16,54,30.0
1,16,54,30,16,54,30.0
2,16,54,37,16,54,30.0
3,16,54,37,16,54,30.0
4,16,54,40,16,54,45.0
...,...,...,...,...,...,...
61501,20,30,52,20,30,45.0
61502,20,30,53,20,30,60.0
61503,20,30,53,20,30,60.0
61505,20,30,54,20,30,60.0


In [ ]:
time_time.loc[time_time["second"] == 60.0,"minute"] = time_time["minute"] + 1
time_time.loc[time_time["second"] == 60.0,"second"] = time_time["second"] -60
time_time.loc[time_time["minute"] == 60.0,"hour"] = time_time["hour"] + 1
time_time.loc[time_time["minute"] == 60.0,"minute"] = time_time["minute"] -60
time_time.loc[time_time["second"] == 0.0,"second"] = "00"
time_time.loc[time_time["second"] == 15.0,"second"] = "15"
time_time.loc[time_time["second"] == 30.0,"second"] = "30"
time_time.loc[time_time["second"] == 45.0,"second"] = "45"

In [ ]:
time_time["hour"] = time_time["hour"].apply(lambda x: str(x))
time_time["minute"] = time_time["minute"].apply(lambda x: str(x))
time_time["second"] = time_time["second"].apply(lambda x: str(x))

In [ ]:
time_time = pd.Series("2021/08/15 " + time_time["hour"] + ":" + time_time["minute"] + ":" + time_time["second"])
time_time = time_time.apply(lambda x: str(x)).copy()

In [ ]:
def converttime(dstring):
    return datetime.strptime(dstring, "%Y/%m/%d %H:%M:%S")

In [ ]:
time_time = time_time.apply(lambda x: converttime(x)).copy() #non so togliere warning copy
df_clean["disc_time"] = time_time
df_clean.head(15)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,created_at,user_id,username,screen_name,text,full_text,hashtags,followers,text_clean,has,hashtag_fin,compl_hashtags,Keep,time,disc_time
0,2021-08-15 14:54:30,69871991,James,JimmyBronze,There will be positives today whatever happens...,,[thfc],263,There will be positives today whatever happens...,[thfc],"[thfc, thfc]",[thfc],Yes,2021-08-15 16:54:30,2021-08-15 16:54:30
1,2021-08-15 14:54:30,85367934,Paul Mckenzie,MaccaTHFC,"@TheSpursgirl @thfc @ManCity Hey Jane, where a...","@TheSpursgirl @thfc @ManCity Hey Jane, where a...",[],565,"@TheSpursgirl @thfc @ManCity Hey Jane, where a...","[Spurs, COYS, thfc]","[Spurs, COYS, thfc]","[COYS, Spurs, thfc]",Yes,2021-08-15 16:54:30,2021-08-15 16:54:30
2,2021-08-15 14:54:37,27957972,Rob Aycock,RobAycock,Nuno is sending a strong message by leaving bo...,Nuno is sending a strong message by leaving bo...,[],393,Nuno is sending a strong message by leaving bo...,"[COYS, THFC]","[COYS, THFC]","[COYS, THFC]",Yes,2021-08-15 16:54:37,2021-08-15 16:54:30
3,2021-08-15 14:54:37,376333378,🍌⚽️🤩 kev bananaman 🍌⚽️🤩,BananamanKev,RT @FreeBetsFooty: 🚨 GIVEAWAY 🚨\n\nIf Son Heun...,,[THFC],1943,RT @FreeBetsFooty: 🚨 GIVEAWAY 🚨\n\nIf Son Heun...,[THFC],"[THFC, THFC]",[THFC],Yes,2021-08-15 16:54:37,2021-08-15 16:54:30
4,2021-08-15 14:54:40,1375839513906278403,なおやん,naoyanyan57,RT @thespursweb: 🚨 NEW: Harry Kane was NOT on ...,,[THFC],287,RT @thespursweb: 🚨 NEW: Harry Kane was NOT on ...,[THFC],"[THFC, THFC]",[THFC],Yes,2021-08-15 16:54:40,2021-08-15 16:54:45
5,2021-08-15 14:54:42,1126381076,AJS 🇮🇱,ajsCOYS,#COYS no pressure x,,[COYS],563,#COYS no pressure x,[COYS],"[COYS, COYS]",[COYS],Yes,2021-08-15 16:54:42,2021-08-15 16:54:45
6,2021-08-15 14:54:44,227448013,thejondaniels,thejondaniels,#COYS Kane was never gonna be in the squad. B...,#COYS Kane was never gonna be in the squad. B...,[COYS],2308,#COYS Kane was never gonna be in the squad. B...,[COYS],"[COYS, COYS]",[COYS],Yes,2021-08-15 16:54:44,2021-08-15 16:54:45
7,2021-08-15 14:54:45,1117479778792030208,🐐 𝓶𝓻.𝓓𝓘𝓔𝓡 🍑,ESTRELLA1882s,RT @thespursweb: 🎶 I want to be in that number...,,[COYS],151,RT @thespursweb: 🎶 I want to be in that number...,[COYS],"[COYS, COYS]",[COYS],Yes,2021-08-15 16:54:45,2021-08-15 16:54:45
8,2021-08-15 14:54:45,1061554145172238336,もなお☃️,monao0911,RT @thespursweb: 🎶 I want to be in that number...,,[COYS],409,RT @thespursweb: 🎶 I want to be in that number...,[COYS],"[COYS, COYS]",[COYS],Yes,2021-08-15 16:54:45,2021-08-15 16:54:45
11,2021-08-15 14:54:49,91628625,SNWDN ❄️ COYS,Liamsnowdon,RT @thespursweb: 🎶 I want to be in that number...,,[COYS],806,RT @thespursweb: 🎶 I want to be in that number...,[COYS],"[COYS, COYS]",[COYS],Yes,2021-08-15 16:54:49,2021-08-15 16:54:45


Va riportato adesso l'orario al riferimento temporale nella partita ovvero il minuto del match.

La partita è iniziata puntualmente alle 17:30, sono stati concessi 2 minuti di recupero nel primo tempo e 6 nel secondo. L'intervallo è durato regolarmente circa 15 minuti. Si ipotizza, inoltre, un compenso di circa un minuto tra evento della partita e pubblicazione del tweet. La durata dell'evento incluso l'intervallo è stata di 113 minuti.

Si fa una prima discrimanzione tra tweet pubblicati durante o a ridosso della partita e gli altri. 

In [ ]:
def minute_from_start(loc_date_time):
    
    date_time = loc_date_time - timedelta(hours = 17, minutes = 31)
    
    if date_time.day == 14: # escludo pre-partita
        minute = np.nan
    
    elif date_time.hour == 0:
        minute = date_time.minute
         
    else: 
        minute =  date_time.minute + date_time.hour*60
        
    return minute

In [ ]:
df_clean['minute_from_start'] = df_clean['disc_time'].apply(minute_from_start)
df_clean.tail(15)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,created_at,user_id,username,screen_name,text,full_text,hashtags,followers,text_clean,has,hashtag_fin,compl_hashtags,Keep,time,disc_time,minute_from_start
61486,2021-08-15 18:30:42,1413132170441363461,The Original Veal Gangster #LFC #F1,GangsterVeal,RT @JackFree_3: Souness 😂 Man dying to talk ab...,,"[LFC, TOTMCI, SuperSunday]",392,RT @JackFree_3: Souness 😂 Man dying to talk ab...,[TOTMCI],"[LFC, TOTMCI, SuperSunday, TOTMCI]","[LFC, SuperSunday, TOTMCI]",Yes,2021-08-15 20:30:42,2021-08-15 20:30:45,179.0
61487,2021-08-15 18:30:42,860904448343891972,Evelyn Arsenal Nakayi,Evelyn_Nakayi,"RT @premierleague: One round down, 37 to go. \...",,[PL],86408,"RT @premierleague: One round down, 37 to go. \...",[PL],"[PL, PL]",[PL],Yes,2021-08-15 20:30:42,2021-08-15 20:30:45,179.0
61488,2021-08-15 18:30:44,842313161982238720,waru ke,waru254,RT @premierleague: 🐝🇨🇴 @WatfordFC’s Cucho Hern...,,[PL],7247,RT @premierleague: 🐝🇨🇴 @WatfordFC’s Cucho Hern...,[PL],"[PL, PL]",[PL],Yes,2021-08-15 20:30:44,2021-08-15 20:30:45,179.0
61489,2021-08-15 18:30:44,900103184592642051,Mugisha Peter,Mugisha93586265,"RT @premierleague: One round down, 37 to go. \...",,[PL],1299,"RT @premierleague: One round down, 37 to go. \...",[PL],"[PL, PL]",[PL],Yes,2021-08-15 20:30:44,2021-08-15 20:30:45,179.0
61492,2021-08-15 18:30:48,4802668096,💲Ice Morgan⚽🎹🎮,norteyrichard62,"RT @premierleague: One round down, 37 to go. \...",,[PL],794,"RT @premierleague: One round down, 37 to go. \...",[PL],"[PL, PL]",[PL],Yes,2021-08-15 20:30:48,2021-08-15 20:30:45,179.0
61495,2021-08-15 18:30:49,1200711480238325760,TheOneOnline,gjohanssons951,"RT @premierleague: One round down, 37 to go. \...",,[PL],8,"RT @premierleague: One round down, 37 to go. \...",[PL],"[PL, PL]",[PL],Yes,2021-08-15 20:30:49,2021-08-15 20:30:45,179.0
61497,2021-08-15 18:30:52,871721315719413760,Nox 🦁💙,nox_manele,"RT @premierleague: One round down, 37 to go. \...",,[PL],1090,"RT @premierleague: One round down, 37 to go. \...",[PL],"[PL, PL]",[PL],Yes,2021-08-15 20:30:52,2021-08-15 20:30:45,179.0
61498,2021-08-15 18:30:52,2841320244,Chase Oden,chase_oden,"RT @mralexthfc: “Are you watching, are you wat...",,"[COYS, THFC, TOTMCI]",124,"RT @mralexthfc: “Are you watching, are you wat...","[COYS, THFC, TOTMCI]","[COYS, THFC, TOTMCI, COYS, THFC, TOTMCI]","[COYS, THFC, TOTMCI]",Yes,2021-08-15 20:30:52,2021-08-15 20:30:45,179.0
61499,2021-08-15 18:30:52,97701828,Ben Haines,benhainess,So much time for Michael Dawson just chilling ...,So much time for Michael Dawson just chilling ...,[],11810,So much time for Michael Dawson just chilling ...,[TOTMCI],[TOTMCI],[TOTMCI],Yes,2021-08-15 20:30:52,2021-08-15 20:30:45,179.0
61500,2021-08-15 18:30:52,1022007188091490304,Flyer🐆🇿🇼🇿🇼🇿🇼,CMutyoza,"RT @BSchweinsteiger: Go for it, Red Devils 💪🏼 ...",,"[MUFC, PremierLeague]",1441,"RT @BSchweinsteiger: Go for it, Red Devils 💪🏼 ...",[PremierLeague],"[MUFC, PremierLeague, PremierLeague]","[MUFC, PremierLeague]",Yes,2021-08-15 20:30:52,2021-08-15 20:30:45,179.0


Suddivido i tweet in Pre-partita, Intervallo, Primo Tempo, Secondo Tempo e Post-Partita

In [ ]:
def pre_partita(minute):
  try:
      if np.isnan(minute) == True:
        moment = "Pre"
      else:
        moment = "Other"
  except:
    moment = "Other"
  return moment

In [ ]:
df_clean['Moment'] = df_clean['minute_from_start'].apply(pre_partita)
df_clean.head(15)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,created_at,user_id,username,screen_name,text,full_text,hashtags,followers,text_clean,has,hashtag_fin,compl_hashtags,Keep,time,disc_time,minute_from_start,Moment
0,2021-08-15 14:54:30,69871991,James,JimmyBronze,There will be positives today whatever happens...,,[thfc],263,There will be positives today whatever happens...,[thfc],"[thfc, thfc]",[thfc],Yes,2021-08-15 16:54:30,2021-08-15 16:54:30,NaN,Pre
1,2021-08-15 14:54:30,85367934,Paul Mckenzie,MaccaTHFC,"@TheSpursgirl @thfc @ManCity Hey Jane, where a...","@TheSpursgirl @thfc @ManCity Hey Jane, where a...",[],565,"@TheSpursgirl @thfc @ManCity Hey Jane, where a...","[Spurs, COYS, thfc]","[Spurs, COYS, thfc]","[COYS, Spurs, thfc]",Yes,2021-08-15 16:54:30,2021-08-15 16:54:30,NaN,Pre
2,2021-08-15 14:54:37,27957972,Rob Aycock,RobAycock,Nuno is sending a strong message by leaving bo...,Nuno is sending a strong message by leaving bo...,[],393,Nuno is sending a strong message by leaving bo...,"[COYS, THFC]","[COYS, THFC]","[COYS, THFC]",Yes,2021-08-15 16:54:37,2021-08-15 16:54:30,NaN,Pre
3,2021-08-15 14:54:37,376333378,🍌⚽️🤩 kev bananaman 🍌⚽️🤩,BananamanKev,RT @FreeBetsFooty: 🚨 GIVEAWAY 🚨\n\nIf Son Heun...,,[THFC],1943,RT @FreeBetsFooty: 🚨 GIVEAWAY 🚨\n\nIf Son Heun...,[THFC],"[THFC, THFC]",[THFC],Yes,2021-08-15 16:54:37,2021-08-15 16:54:30,NaN,Pre
4,2021-08-15 14:54:40,1375839513906278403,なおやん,naoyanyan57,RT @thespursweb: 🚨 NEW: Harry Kane was NOT on ...,,[THFC],287,RT @thespursweb: 🚨 NEW: Harry Kane was NOT on ...,[THFC],"[THFC, THFC]",[THFC],Yes,2021-08-15 16:54:40,2021-08-15 16:54:45,NaN,Pre
5,2021-08-15 14:54:42,1126381076,AJS 🇮🇱,ajsCOYS,#COYS no pressure x,,[COYS],563,#COYS no pressure x,[COYS],"[COYS, COYS]",[COYS],Yes,2021-08-15 16:54:42,2021-08-15 16:54:45,NaN,Pre
6,2021-08-15 14:54:44,227448013,thejondaniels,thejondaniels,#COYS Kane was never gonna be in the squad. B...,#COYS Kane was never gonna be in the squad. B...,[COYS],2308,#COYS Kane was never gonna be in the squad. B...,[COYS],"[COYS, COYS]",[COYS],Yes,2021-08-15 16:54:44,2021-08-15 16:54:45,NaN,Pre
7,2021-08-15 14:54:45,1117479778792030208,🐐 𝓶𝓻.𝓓𝓘𝓔𝓡 🍑,ESTRELLA1882s,RT @thespursweb: 🎶 I want to be in that number...,,[COYS],151,RT @thespursweb: 🎶 I want to be in that number...,[COYS],"[COYS, COYS]",[COYS],Yes,2021-08-15 16:54:45,2021-08-15 16:54:45,NaN,Pre
8,2021-08-15 14:54:45,1061554145172238336,もなお☃️,monao0911,RT @thespursweb: 🎶 I want to be in that number...,,[COYS],409,RT @thespursweb: 🎶 I want to be in that number...,[COYS],"[COYS, COYS]",[COYS],Yes,2021-08-15 16:54:45,2021-08-15 16:54:45,NaN,Pre
11,2021-08-15 14:54:49,91628625,SNWDN ❄️ COYS,Liamsnowdon,RT @thespursweb: 🎶 I want to be in that number...,,[COYS],806,RT @thespursweb: 🎶 I want to be in that number...,[COYS],"[COYS, COYS]",[COYS],Yes,2021-08-15 16:54:49,2021-08-15 16:54:45,NaN,Pre


In [ ]:
pre_match = df_clean[df_clean["Moment"] == "Pre"].copy()
match_post = df_clean[df_clean["Moment"] == "Other"].copy()
match_post.head(15)

,created_at,user_id,username,screen_name,text,full_text,hashtags,followers,text_clean,has,hashtag_fin,compl_hashtags,Keep,time,disc_time,minute_from_start,Moment
1777,2021-08-15 15:30:53,576167561,far,untriplesix,COME ON BOYS!! #COYS https://t.co/126b7D5vQq,,[COYS],326,COME ON BOYS!! #COYS https://t.co/126b7D5vQq,[COYS],"[COYS, COYS]",[COYS],Yes,2021-08-15 17:30:53,2021-08-15 17:31:00,0.0,Other
1778,2021-08-15 15:30:53,266251364,LJohnson,lazqetjohnson,"Our fans are the best in the England, by a cou...",,[COYS],24053,"Our fans are the best in the England, by a cou...",[COYS],"[COYS, COYS]",[COYS],Yes,2021-08-15 17:30:53,2021-08-15 17:31:00,0.0,Other
1779,2021-08-15 15:30:58,276233686,ryan 🇨🇦,RyanDHobart,Anyone know who that woman is who just straigh...,,[coys],635,Anyone know who that woman is who just straigh...,[coys],"[coys, coys]",[coys],Yes,2021-08-15 17:30:58,2021-08-15 17:31:00,0.0,Other
1780,2021-08-15 15:30:58,161413417,Colin,colinblouin,Jack Grealish playing for Man City be like... ...,,[COYS],662,Jack Grealish playing for Man City be like... ...,[COYS],"[COYS, COYS]",[COYS],Yes,2021-08-15 17:30:58,2021-08-15 17:31:00,0.0,Other
1781,2021-08-15 15:31:00,1257180002,伊沢拓司,tax_i_,Come on you Spurs!!\n#COYS,,[COYS],449946,Come on you Spurs!!\n#COYS,[COYS],"[COYS, COYS]",[COYS],Yes,2021-08-15 17:31:00,2021-08-15 17:31:00,0.0,Other
1782,2021-08-15 15:31:00,24518986,littlejason,littlejason,Get it done boys !! #COYS,,[COYS],701,Get it done boys !! #COYS,[COYS],"[COYS, COYS]",[COYS],Yes,2021-08-15 17:31:00,2021-08-15 17:31:00,0.0,Other
1783,2021-08-15 15:31:00,338670340,Richard Clark III,rkc3indy,Matchday 1 #EPL #THFC #COYS #FuckManShitty @ B...,,"[EPL, THFC, COYS, FuckManShitty]",454,Matchday 1 #EPL #THFC #COYS #FuckManShitty @ B...,"[COYS, THFC]","[EPL, THFC, COYS, FuckManShitty, COYS, THFC]","[COYS, EPL, FuckManShitty, THFC]",Yes,2021-08-15 17:31:00,2021-08-15 17:31:00,0.0,Other
1785,2021-08-15 15:31:02,1287748779324641290,Bez,Bez_FPL,Come on!!!! #COYS,,[COYS],662,Come on!!!! #COYS,[COYS],"[COYS, COYS]",[COYS],Yes,2021-08-15 17:31:02,2021-08-15 17:31:00,0.0,Other
1786,2021-08-15 15:31:02,1311955295686492163,😈💋hellz2780💋😈,HelenUpsher,Watching \n\nTottenham v Man C\n#Tottenham,,[Tottenham],339,Watching \n\nTottenham v Man C\n#Tottenham,[Tottenham],"[Tottenham, Tottenham]",[Tottenham],Yes,2021-08-15 17:31:02,2021-08-15 17:31:00,0.0,Other
1788,2021-08-15 15:31:03,1385194605365321729,DP_34,DPT_34,Even Levys wife has had enough of him #COYS #THFC,,"[COYS, THFC]",94,Even Levys wife has had enough of him #COYS #THFC,"[COYS, THFC]","[COYS, THFC, COYS, THFC]","[COYS, THFC]",Yes,2021-08-15 17:31:03,2021-08-15 17:31:00,0.0,Other


In [ ]:
len(pre_match) + len(match_post) == len(df_clean)

True

In [ ]:
def momento(minute):
  if minute <= 46:
    moment = "Primo"
  elif minute > 112:
    moment = "Post"  
  elif (minute > 46) & (minute <= 61):
    moment = "Intervallo"
  else:
    moment = "Secondo"
  
  return moment

In [ ]:
match_post['Moment'] = match_post['minute_from_start'].apply(momento)
match_post.head(15)

,created_at,user_id,username,screen_name,text,full_text,hashtags,followers,text_clean,has,hashtag_fin,compl_hashtags,Keep,time,disc_time,minute_from_start,Moment
1777,2021-08-15 15:30:53,576167561,far,untriplesix,COME ON BOYS!! #COYS https://t.co/126b7D5vQq,,[COYS],326,COME ON BOYS!! #COYS https://t.co/126b7D5vQq,[COYS],"[COYS, COYS]",[COYS],Yes,2021-08-15 17:30:53,2021-08-15 17:31:00,0.0,Primo
1778,2021-08-15 15:30:53,266251364,LJohnson,lazqetjohnson,"Our fans are the best in the England, by a cou...",,[COYS],24053,"Our fans are the best in the England, by a cou...",[COYS],"[COYS, COYS]",[COYS],Yes,2021-08-15 17:30:53,2021-08-15 17:31:00,0.0,Primo
1779,2021-08-15 15:30:58,276233686,ryan 🇨🇦,RyanDHobart,Anyone know who that woman is who just straigh...,,[coys],635,Anyone know who that woman is who just straigh...,[coys],"[coys, coys]",[coys],Yes,2021-08-15 17:30:58,2021-08-15 17:31:00,0.0,Primo
1780,2021-08-15 15:30:58,161413417,Colin,colinblouin,Jack Grealish playing for Man City be like... ...,,[COYS],662,Jack Grealish playing for Man City be like... ...,[COYS],"[COYS, COYS]",[COYS],Yes,2021-08-15 17:30:58,2021-08-15 17:31:00,0.0,Primo
1781,2021-08-15 15:31:00,1257180002,伊沢拓司,tax_i_,Come on you Spurs!!\n#COYS,,[COYS],449946,Come on you Spurs!!\n#COYS,[COYS],"[COYS, COYS]",[COYS],Yes,2021-08-15 17:31:00,2021-08-15 17:31:00,0.0,Primo
1782,2021-08-15 15:31:00,24518986,littlejason,littlejason,Get it done boys !! #COYS,,[COYS],701,Get it done boys !! #COYS,[COYS],"[COYS, COYS]",[COYS],Yes,2021-08-15 17:31:00,2021-08-15 17:31:00,0.0,Primo
1783,2021-08-15 15:31:00,338670340,Richard Clark III,rkc3indy,Matchday 1 #EPL #THFC #COYS #FuckManShitty @ B...,,"[EPL, THFC, COYS, FuckManShitty]",454,Matchday 1 #EPL #THFC #COYS #FuckManShitty @ B...,"[COYS, THFC]","[EPL, THFC, COYS, FuckManShitty, COYS, THFC]","[COYS, EPL, FuckManShitty, THFC]",Yes,2021-08-15 17:31:00,2021-08-15 17:31:00,0.0,Primo
1785,2021-08-15 15:31:02,1287748779324641290,Bez,Bez_FPL,Come on!!!! #COYS,,[COYS],662,Come on!!!! #COYS,[COYS],"[COYS, COYS]",[COYS],Yes,2021-08-15 17:31:02,2021-08-15 17:31:00,0.0,Primo
1786,2021-08-15 15:31:02,1311955295686492163,😈💋hellz2780💋😈,HelenUpsher,Watching \n\nTottenham v Man C\n#Tottenham,,[Tottenham],339,Watching \n\nTottenham v Man C\n#Tottenham,[Tottenham],"[Tottenham, Tottenham]",[Tottenham],Yes,2021-08-15 17:31:02,2021-08-15 17:31:00,0.0,Primo
1788,2021-08-15 15:31:03,1385194605365321729,DP_34,DPT_34,Even Levys wife has had enough of him #COYS #THFC,,"[COYS, THFC]",94,Even Levys wife has had enough of him #COYS #THFC,"[COYS, THFC]","[COYS, THFC, COYS, THFC]","[COYS, THFC]",Yes,2021-08-15 17:31:03,2021-08-15 17:31:00,0.0,Primo


In [ ]:
df_time = pd.concat([match_post, pre_match], ignore_index=True).copy()
df_time

,created_at,user_id,username,screen_name,text,full_text,hashtags,followers,text_clean,has,hashtag_fin,compl_hashtags,Keep,time,disc_time,minute_from_start,Moment
0,2021-08-15 15:30:53,576167561,far,untriplesix,COME ON BOYS!! #COYS https://t.co/126b7D5vQq,,[COYS],326,COME ON BOYS!! #COYS https://t.co/126b7D5vQq,[COYS],"[COYS, COYS]",[COYS],Yes,2021-08-15 17:30:53,2021-08-15 17:31:00,0.0,Primo
1,2021-08-15 15:30:53,266251364,LJohnson,lazqetjohnson,"Our fans are the best in the England, by a cou...",,[COYS],24053,"Our fans are the best in the England, by a cou...",[COYS],"[COYS, COYS]",[COYS],Yes,2021-08-15 17:30:53,2021-08-15 17:31:00,0.0,Primo
2,2021-08-15 15:30:58,276233686,ryan 🇨🇦,RyanDHobart,Anyone know who that woman is who just straigh...,,[coys],635,Anyone know who that woman is who just straigh...,[coys],"[coys, coys]",[coys],Yes,2021-08-15 17:30:58,2021-08-15 17:31:00,0.0,Primo
3,2021-08-15 15:30:58,161413417,Colin,colinblouin,Jack Grealish playing for Man City be like... ...,,[COYS],662,Jack Grealish playing for Man City be like... ...,[COYS],"[COYS, COYS]",[COYS],Yes,2021-08-15 17:30:58,2021-08-15 17:31:00,0.0,Primo
4,2021-08-15 15:31:00,1257180002,伊沢拓司,tax_i_,Come on you Spurs!!\n#COYS,,[COYS],449946,Come on you Spurs!!\n#COYS,[COYS],"[COYS, COYS]",[COYS],Yes,2021-08-15 17:31:00,2021-08-15 17:31:00,0.0,Primo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48393,2021-08-15 15:30:50,1251407314780934144,Anto tha Dj,Dj_razer_ke,RT @TvLive66: #TOTMCI LIVE STREAM\nTottenham v...,,[TOTMCI],2068,RT @TvLive66: #TOTMCI LIVE STREAM\nTottenham v...,[TOTMCI],"[TOTMCI, TOTMCI]",[TOTMCI],Yes,2021-08-15 17:30:50,2021-08-15 17:30:45,NaN,Pre
48394,2021-08-15 15:30:51,1302750688204795912,RizzoX8,rizzo_x8,"Tell us you’re a small club, without actually ...",,[TOTMCI],13,"Tell us you’re a small club, without actually ...",[TOTMCI],"[TOTMCI, TOTMCI]",[TOTMCI],Yes,2021-08-15 17:30:51,2021-08-15 17:30:45,NaN,Pre
48395,2021-08-15 15:30:51,1227274069990506497,Andres🦧,zRadiqtion,COYS #TOTMCI,,[TOTMCI],407,COYS #TOTMCI,[TOTMCI],"[TOTMCI, TOTMCI]",[TOTMCI],Yes,2021-08-15 17:30:51,2021-08-15 17:30:45,NaN,Pre
48396,2021-08-15 15:30:52,1232674609377488897,Mr. Koolest 🥷🏿,___STxN,not my girlfriend wanting me to base her scalp...,,[TOTMCI],6367,not my girlfriend wanting me to base her scalp...,[TOTMCI],"[TOTMCI, TOTMCI]",[TOTMCI],Yes,2021-08-15 17:30:52,2021-08-15 17:30:45,NaN,Pre


Va costruita una funzione che attribuisca il minuto all'interno della partita.

In [ ]:
def match_minute(row):
  st_minute = row['minute_from_start']
  moment = row['Moment']
  
  if moment == "Secondo":
    minuto = st_minute - 17 + 1
  
  elif moment == "Primo":
    minuto = st_minute + 1
  
  else:
    minuto = np.nan
  
  return minuto

In [ ]:
df_time['Minute'] = df_time.apply(match_minute, axis = 1)
df_time[df_time['Minute'] == 47]

,created_at,user_id,username,screen_name,text,full_text,hashtags,followers,text_clean,has,hashtag_fin,compl_hashtags,Keep,time,disc_time,minute_from_start,Moment,Minute
954,2021-08-15 16:16:53,35783746,(((Don_ THFC))),Dnannery,Ended up a good half #COYS,,[COYS],391,Ended up a good half #COYS,[COYS],"[COYS, COYS]",[COYS],Yes,2021-08-15 18:16:53,2021-08-15 18:17:00,46.0,Primo,47.0
955,2021-08-15 16:16:54,309243917,John,Ramma_____,Little wonder Harry Kane wants away if he can’...,,[THFC],4016,Little wonder Harry Kane wants away if he can’...,[THFC],"[THFC, THFC]",[THFC],Yes,2021-08-15 18:16:54,2021-08-15 18:17:00,46.0,Primo,47.0
956,2021-08-15 16:16:57,1353358695162327043,George,thfcgeorge0,RT @TottenhamTiers: #thfc striker Harry Kane h...,,[thfc],86,RT @TottenhamTiers: #thfc striker Harry Kane h...,[thfc],"[thfc, thfc]",[thfc],Yes,2021-08-15 18:16:57,2021-08-15 18:17:00,46.0,Primo,47.0
957,2021-08-15 16:16:57,2447976632,Zain,maestroZain,"I'm happy so far, we have been the better team...",,[COYS],46,"I'm happy so far, we have been the better team...",[COYS],"[COYS, COYS]",[COYS],Yes,2021-08-15 18:16:57,2021-08-15 18:17:00,46.0,Primo,47.0
958,2021-08-15 16:16:57,1028987392701673472,Hotspurs News Alerts,AlertsHotspur,LATEST: Shot H Son (40) Tottenham 0 - 0 Manche...,,[THFC],1872,LATEST: Shot H Son (40) Tottenham 0 - 0 Manche...,[THFC],"[THFC, THFC]",[THFC],Yes,2021-08-15 18:16:57,2021-08-15 18:17:00,46.0,Primo,47.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23526,2021-08-15 16:34:48,249303999,E l l a 🥶,EllaMarie_94,Son to score and Fernandinho to be carded pls ...,,[TOTMCI],1065,Son to score and Fernandinho to be carded pls ...,[TOTMCI],"[TOTMCI, TOTMCI]",[TOTMCI],Yes,2021-08-15 18:34:48,2021-08-15 18:34:45,63.0,Secondo,47.0
23527,2021-08-15 16:34:50,800338844,Rnjn ⭐ ⭐,IAMRnjn,Jack Grealish first half highlights for Manche...,,[TOTMCI],795,Jack Grealish first half highlights for Manche...,[TOTMCI],"[TOTMCI, TOTMCI]",[TOTMCI],Yes,2021-08-15 18:34:50,2021-08-15 18:34:45,63.0,Secondo,47.0
23528,2021-08-15 16:34:51,3288676096,रमेश कान्छो,Ramesh_kanchho,RT @UCL78284507: Tottenham Hotspur vs Manchest...,,[TOTMCI],3832,RT @UCL78284507: Tottenham Hotspur vs Manchest...,[TOTMCI],"[TOTMCI, TOTMCI]",[TOTMCI],Yes,2021-08-15 18:34:51,2021-08-15 18:34:45,63.0,Secondo,47.0
23529,2021-08-15 16:34:52,138044002,Maybe: Aaron,doubleatho,RT @terryflewers: We are LIVE at the final whi...,,[TOTMCI],985,RT @terryflewers: We are LIVE at the final whi...,[TOTMCI],"[TOTMCI, TOTMCI]",[TOTMCI],Yes,2021-08-15 18:34:52,2021-08-15 18:34:45,63.0,Secondo,47.0


Con la seguente funzione si gestisce la questione minuti di recupero creando riferimenti univoci per il minuto della partita nel dataset.

In [ ]:
def minute_str(row):
  
  minute = row['Minute']
  moment = row['Moment']
  
  if moment == "Primo":
    if minute <= 45:
      min = str(int(minute))
    else:
      rec = int(minute - 45)
      min = "45+"+str(rec)
  
  elif moment == "Secondo":
    if minute <= 90:
      min = str(int(minute))
    else:
      rec = int(minute - 90)
      min = "90+"+str(rec)
  
  else:
    min = np.nan
  
  return min

In [ ]:
df_time['Min_Cronaca'] = df_time.apply(minute_str, axis = 1)
df_time

,created_at,user_id,username,screen_name,text,full_text,hashtags,followers,text_clean,has,hashtag_fin,compl_hashtags,Keep,time,disc_time,minute_from_start,Moment,Minute,Min_Cronaca
0,2021-08-15 15:30:53,576167561,far,untriplesix,COME ON BOYS!! #COYS https://t.co/126b7D5vQq,,[COYS],326,COME ON BOYS!! #COYS https://t.co/126b7D5vQq,[COYS],"[COYS, COYS]",[COYS],Yes,2021-08-15 17:30:53,2021-08-15 17:31:00,0.0,Primo,1.0,1
1,2021-08-15 15:30:53,266251364,LJohnson,lazqetjohnson,"Our fans are the best in the England, by a cou...",,[COYS],24053,"Our fans are the best in the England, by a cou...",[COYS],"[COYS, COYS]",[COYS],Yes,2021-08-15 17:30:53,2021-08-15 17:31:00,0.0,Primo,1.0,1
2,2021-08-15 15:30:58,276233686,ryan 🇨🇦,RyanDHobart,Anyone know who that woman is who just straigh...,,[coys],635,Anyone know who that woman is who just straigh...,[coys],"[coys, coys]",[coys],Yes,2021-08-15 17:30:58,2021-08-15 17:31:00,0.0,Primo,1.0,1
3,2021-08-15 15:30:58,161413417,Colin,colinblouin,Jack Grealish playing for Man City be like... ...,,[COYS],662,Jack Grealish playing for Man City be like... ...,[COYS],"[COYS, COYS]",[COYS],Yes,2021-08-15 17:30:58,2021-08-15 17:31:00,0.0,Primo,1.0,1
4,2021-08-15 15:31:00,1257180002,伊沢拓司,tax_i_,Come on you Spurs!!\n#COYS,,[COYS],449946,Come on you Spurs!!\n#COYS,[COYS],"[COYS, COYS]",[COYS],Yes,2021-08-15 17:31:00,2021-08-15 17:31:00,0.0,Primo,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48393,2021-08-15 15:30:50,1251407314780934144,Anto tha Dj,Dj_razer_ke,RT @TvLive66: #TOTMCI LIVE STREAM\nTottenham v...,,[TOTMCI],2068,RT @TvLive66: #TOTMCI LIVE STREAM\nTottenham v...,[TOTMCI],"[TOTMCI, TOTMCI]",[TOTMCI],Yes,2021-08-15 17:30:50,2021-08-15 17:30:45,NaN,Pre,NaN,NaN
48394,2021-08-15 15:30:51,1302750688204795912,RizzoX8,rizzo_x8,"Tell us you’re a small club, without actually ...",,[TOTMCI],13,"Tell us you’re a small club, without actually ...",[TOTMCI],"[TOTMCI, TOTMCI]",[TOTMCI],Yes,2021-08-15 17:30:51,2021-08-15 17:30:45,NaN,Pre,NaN,NaN
48395,2021-08-15 15:30:51,1227274069990506497,Andres🦧,zRadiqtion,COYS #TOTMCI,,[TOTMCI],407,COYS #TOTMCI,[TOTMCI],"[TOTMCI, TOTMCI]",[TOTMCI],Yes,2021-08-15 17:30:51,2021-08-15 17:30:45,NaN,Pre,NaN,NaN
48396,2021-08-15 15:30:52,1232674609377488897,Mr. Koolest 🥷🏿,___STxN,not my girlfriend wanting me to base her scalp...,,[TOTMCI],6367,not my girlfriend wanting me to base her scalp...,[TOTMCI],"[TOTMCI, TOTMCI]",[TOTMCI],Yes,2021-08-15 17:30:52,2021-08-15 17:30:45,NaN,Pre,NaN,NaN


In [ ]:
df_time[df_time["Min_Cronaca"] == "45+2"]

,created_at,user_id,username,screen_name,text,full_text,hashtags,followers,text_clean,has,hashtag_fin,compl_hashtags,Keep,time,disc_time,minute_from_start,Moment,Minute,Min_Cronaca
954,2021-08-15 16:16:53,35783746,(((Don_ THFC))),Dnannery,Ended up a good half #COYS,,[COYS],391,Ended up a good half #COYS,[COYS],"[COYS, COYS]",[COYS],Yes,2021-08-15 18:16:53,2021-08-15 18:17:00,46.0,Primo,47.0,45+2
955,2021-08-15 16:16:54,309243917,John,Ramma_____,Little wonder Harry Kane wants away if he can’...,,[THFC],4016,Little wonder Harry Kane wants away if he can’...,[THFC],"[THFC, THFC]",[THFC],Yes,2021-08-15 18:16:54,2021-08-15 18:17:00,46.0,Primo,47.0,45+2
956,2021-08-15 16:16:57,1353358695162327043,George,thfcgeorge0,RT @TottenhamTiers: #thfc striker Harry Kane h...,,[thfc],86,RT @TottenhamTiers: #thfc striker Harry Kane h...,[thfc],"[thfc, thfc]",[thfc],Yes,2021-08-15 18:16:57,2021-08-15 18:17:00,46.0,Primo,47.0,45+2
957,2021-08-15 16:16:57,2447976632,Zain,maestroZain,"I'm happy so far, we have been the better team...",,[COYS],46,"I'm happy so far, we have been the better team...",[COYS],"[COYS, COYS]",[COYS],Yes,2021-08-15 18:16:57,2021-08-15 18:17:00,46.0,Primo,47.0,45+2
958,2021-08-15 16:16:57,1028987392701673472,Hotspurs News Alerts,AlertsHotspur,LATEST: Shot H Son (40) Tottenham 0 - 0 Manche...,,[THFC],1872,LATEST: Shot H Son (40) Tottenham 0 - 0 Manche...,[THFC],"[THFC, THFC]",[THFC],Yes,2021-08-15 18:16:57,2021-08-15 18:17:00,46.0,Primo,47.0,45+2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21829,2021-08-15 16:17:52,1374834173660303363,Thesharecapitalist,Thesharecapita1,That first half shows why City need Kane they ...,,[TOTMCI],27,That first half shows why City need Kane they ...,[TOTMCI],"[TOTMCI, TOTMCI]",[TOTMCI],Yes,2021-08-15 18:17:52,2021-08-15 18:17:45,46.0,Primo,47.0,45+2
21830,2021-08-15 16:17:52,1149698188909142016,Alex Mugo🇰🇪,AlexMug89426813,RT @Eplhere: #TOTMCI LIVE STREAM\nTottenham vs...,,[TOTMCI],224,RT @Eplhere: #TOTMCI LIVE STREAM\nTottenham vs...,[TOTMCI],"[TOTMCI, TOTMCI]",[TOTMCI],Yes,2021-08-15 18:17:52,2021-08-15 18:17:45,46.0,Primo,47.0,45+2
21831,2021-08-15 16:17:52,1068857605349507074,Ahmaddhani69_,Xhunters11,RT @TvLive66: #TOTMCI LIVE STREAM\nTottenham v...,,[TOTMCI],0,RT @TvLive66: #TOTMCI LIVE STREAM\nTottenham v...,[TOTMCI],"[TOTMCI, TOTMCI]",[TOTMCI],Yes,2021-08-15 18:17:52,2021-08-15 18:17:45,46.0,Primo,47.0,45+2
21832,2021-08-15 16:17:52,1014104889268621312,RedDevilBoi,0d1nR,Reminder. City actually paid £100 million for ...,,"[PremierLeague, TOTMCI]",56,Reminder. City actually paid £100 million for ...,"[PremierLeague, TOTMCI]","[PremierLeague, TOTMCI, PremierLeague, TOTMCI]","[PremierLeague, TOTMCI]",Yes,2021-08-15 18:17:52,2021-08-15 18:17:45,46.0,Primo,47.0,45+2


In [ ]:
df_T_clean = df_time.drop(['Minute', 'created_at', 'Keep', 'hashtag_fin','has', 'text', 'full_text', 'username', 'disc_time'], axis = 1)
df_T_clean

,user_id,screen_name,hashtags,followers,text_clean,compl_hashtags,time,minute_from_start,Moment,Min_Cronaca
0,576167561,untriplesix,[COYS],326,COME ON BOYS!! #COYS https://t.co/126b7D5vQq,[COYS],2021-08-15 17:30:53,0.0,Primo,1
1,266251364,lazqetjohnson,[COYS],24053,"Our fans are the best in the England, by a cou...",[COYS],2021-08-15 17:30:53,0.0,Primo,1
2,276233686,RyanDHobart,[coys],635,Anyone know who that woman is who just straigh...,[coys],2021-08-15 17:30:58,0.0,Primo,1
3,161413417,colinblouin,[COYS],662,Jack Grealish playing for Man City be like... ...,[COYS],2021-08-15 17:30:58,0.0,Primo,1
4,1257180002,tax_i_,[COYS],449946,Come on you Spurs!!\n#COYS,[COYS],2021-08-15 17:31:00,0.0,Primo,1
...,...,...,...,...,...,...,...,...,...,...
48393,1251407314780934144,Dj_razer_ke,[TOTMCI],2068,RT @TvLive66: #TOTMCI LIVE STREAM\nTottenham v...,[TOTMCI],2021-08-15 17:30:50,NaN,Pre,NaN
48394,1302750688204795912,rizzo_x8,[TOTMCI],13,"Tell us you’re a small club, without actually ...",[TOTMCI],2021-08-15 17:30:51,NaN,Pre,NaN
48395,1227274069990506497,zRadiqtion,[TOTMCI],407,COYS #TOTMCI,[TOTMCI],2021-08-15 17:30:51,NaN,Pre,NaN
48396,1232674609377488897,___STxN,[TOTMCI],6367,not my girlfriend wanting me to base her scalp...,[TOTMCI],2021-08-15 17:30:52,NaN,Pre,NaN


Si procede ad attribuire l'appartenenza ad una tifoseria basandosi sugli hashtag citati e sulla relativa sentiment analysis.

Di seguito l'importazione delle librerie necessarie.

In [ ]:
import nltk
from nltk.stem import PorterStemmer
from gensim.parsing.preprocessing import remove_stopwords
import string
!pip3 install vaderSentiment
import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

     |████████████████████████████████| 125 kB 5.0 MB/s 


Applichiamo una pulizia del testo per migliorare l'efficacia della sentiment analysis.

In [ ]:
df_T_clean["text_clean"] = df_T_clean["text_clean"].apply(lambda x: re.sub(r"RT","",x))
df_T_clean["text_clean"] = df_T_clean["text_clean"].apply(lambda x: re.sub(r":","",x))
df_T_clean["text_clean"] = df_T_clean["text_clean"].apply(lambda x: re.sub("@\w*","",x))
df_T_clean["text_clean"] = df_T_clean["text_clean"].apply(lambda x: re.sub("#\w*","",x))
df_T_clean["text_clean"] = df_T_clean["text_clean"].apply(lambda x: re.sub("\n","",x))
df_T_clean["text_clean"] = df_T_clean["text_clean"].apply(str)
df_T_clean["text_clean"] = df_T_clean["text_clean"].str.lower()
df_T_clean["text_clean"] = df_T_clean["text_clean"].apply(lambda text: re.sub(r"\d+", "",text))
df_T_clean["text_clean"] = df_T_clean["text_clean"].apply(lambda text: text.translate(str.maketrans('', '', string.punctuation)))
df_T_clean["text_clean"] = df_T_clean["text_clean"].apply(lambda text: remove_stopwords(text))
df_T_clean["text_clean"] = df_T_clean["text_clean"].apply(lambda text: re.sub(r"'", "",text))
df_T_clean["text_clean"] = df_T_clean["text_clean"].apply(lambda text: re.sub(r"https\w*", "",text))

Attribuzione degli score per la sentiment analysis.

In [ ]:
sid = SentimentIntensityAnalyzer()
df_T_clean['scores'] = df_T_clean['text_clean'].apply(lambda review: sid.polarity_scores(review))
df_T_clean['compound']  = df_T_clean['scores'].apply(lambda score_dict: score_dict['compound'])
df_T_clean.head(15)

,user_id,screen_name,hashtags,followers,text_clean,compl_hashtags,time,minute_from_start,Moment,Min_Cronaca,scores,compound
0,576167561,untriplesix,[COYS],326,come boys,[COYS],2021-08-15 17:30:53,0.0,Primo,1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
1,266251364,lazqetjohnson,[COYS],24053,fans best england country mile,[COYS],2021-08-15 17:30:53,0.0,Primo,1,"{'neg': 0.0, 'neu': 0.488, 'pos': 0.512, 'comp...",0.6369
2,276233686,RyanDHobart,[coys],635,know woman straight ignored daniel levy shes n...,[coys],2021-08-15 17:30:58,0.0,Primo,1,"{'neg': 0.331, 'neu': 0.508, 'pos': 0.161, 'co...",-0.2500
3,161413417,colinblouin,[COYS],662,jack grealish playing man city like,[COYS],2021-08-15 17:30:58,0.0,Primo,1,"{'neg': 0.0, 'neu': 0.482, 'pos': 0.518, 'comp...",0.5106
4,1257180002,tax_i_,[COYS],449946,come spurs,[COYS],2021-08-15 17:31:00,0.0,Primo,1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
5,24518986,littlejason,[COYS],701,boys,[COYS],2021-08-15 17:31:00,0.0,Primo,1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
6,338670340,rkc3indy,"[EPL, THFC, COYS, FuckManShitty]",454,matchday buffalo wild wings,"[COYS, EPL, FuckManShitty, THFC]",2021-08-15 17:31:00,0.0,Primo,1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
7,1287748779324641290,Bez_FPL,[COYS],662,come,[COYS],2021-08-15 17:31:02,0.0,Primo,1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
8,1311955295686492163,HelenUpsher,[Tottenham],339,watching tottenham v man c,[Tottenham],2021-08-15 17:31:02,0.0,Primo,1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
9,1385194605365321729,DPT_34,"[COYS, THFC]",94,levys wife,"[COYS, THFC]",2021-08-15 17:31:03,0.0,Primo,1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000


Viene attribuito ad ogni tweet una label circa il sentiment del tweet sulla base dello score ottenuto.

In [ ]:
def comp_score(compund):
  if compund >=0.05:
    return 'Positive'
  elif compund <= -0.05:
    return 'Negative'
  else:
    return 'Neutral'

In [ ]:
df_T_clean['comp_score'] = df_T_clean['compound'].apply(comp_score)
df_T_clean.head(15)

,user_id,screen_name,hashtags,followers,text_clean,compl_hashtags,time,minute_from_start,Moment,Min_Cronaca,scores,compound,comp_score
0,576167561,untriplesix,[COYS],326,come boys,[COYS],2021-08-15 17:30:53,0.0,Primo,1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,Neutral
1,266251364,lazqetjohnson,[COYS],24053,fans best england country mile,[COYS],2021-08-15 17:30:53,0.0,Primo,1,"{'neg': 0.0, 'neu': 0.488, 'pos': 0.512, 'comp...",0.6369,Positive
2,276233686,RyanDHobart,[coys],635,know woman straight ignored daniel levy shes n...,[coys],2021-08-15 17:30:58,0.0,Primo,1,"{'neg': 0.331, 'neu': 0.508, 'pos': 0.161, 'co...",-0.2500,Negative
3,161413417,colinblouin,[COYS],662,jack grealish playing man city like,[COYS],2021-08-15 17:30:58,0.0,Primo,1,"{'neg': 0.0, 'neu': 0.482, 'pos': 0.518, 'comp...",0.5106,Positive
4,1257180002,tax_i_,[COYS],449946,come spurs,[COYS],2021-08-15 17:31:00,0.0,Primo,1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,Neutral
5,24518986,littlejason,[COYS],701,boys,[COYS],2021-08-15 17:31:00,0.0,Primo,1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,Neutral
6,338670340,rkc3indy,"[EPL, THFC, COYS, FuckManShitty]",454,matchday buffalo wild wings,"[COYS, EPL, FuckManShitty, THFC]",2021-08-15 17:31:00,0.0,Primo,1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,Neutral
7,1287748779324641290,Bez_FPL,[COYS],662,come,[COYS],2021-08-15 17:31:02,0.0,Primo,1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,Neutral
8,1311955295686492163,HelenUpsher,[Tottenham],339,watching tottenham v man c,[Tottenham],2021-08-15 17:31:02,0.0,Primo,1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,Neutral
9,1385194605365321729,DPT_34,"[COYS, THFC]",94,levys wife,"[COYS, THFC]",2021-08-15 17:31:03,0.0,Primo,1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,Neutral


Si crea una funzione che attribuisce la supposta appartenenza del tweet ad una tifoseria basandosi sugli hashtag citati e sulla neutralità o meno del tweet.

In [ ]:
def tifoso(row):
  compound = row['compound'] 
  riga = row['compl_hashtags']
  if  ([row['comp_score'] =='Neutral']) and (('ManCity' in riga) or ('ManchesterCity' in riga)  or ('mcfc' in riga) or ('MCFC' in riga)) and (('COYS' in riga) or ('thfc' in riga) or ('THFC' in riga) or ('spurs' in riga) or ('Spurs' in riga) or ('Tottenham' in riga)):
      return "Neutral"
  
  elif (('ManCity' in riga) or ('ManchesterCity' in riga)  or ('mcfc' in riga) or ('MCFC' in riga)):
    return "Manchester City" 
  
  elif (('COYS' in riga) or ('thfc' in riga) or ('THFC' in riga) or ('spurs' in riga) or ('Spurs' in riga) or ('Tottenham' in riga)):
    return "Tottenham" 
  
  else:
    return "Neutral"
  

In [ ]:
df_T_clean['tifo'] = df_T_clean.apply(tifoso, axis=1)
df_T_clean.head(15)

,user_id,screen_name,hashtags,followers,text_clean,compl_hashtags,time,minute_from_start,Moment,Min_Cronaca,scores,compound,comp_score,tifo
0,576167561,untriplesix,[COYS],326,come boys,[COYS],2021-08-15 17:30:53,0.0,Primo,1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,Neutral,Tottenham
1,266251364,lazqetjohnson,[COYS],24053,fans best england country mile,[COYS],2021-08-15 17:30:53,0.0,Primo,1,"{'neg': 0.0, 'neu': 0.488, 'pos': 0.512, 'comp...",0.6369,Positive,Tottenham
2,276233686,RyanDHobart,[coys],635,know woman straight ignored daniel levy shes n...,[coys],2021-08-15 17:30:58,0.0,Primo,1,"{'neg': 0.331, 'neu': 0.508, 'pos': 0.161, 'co...",-0.2500,Negative,Neutral
3,161413417,colinblouin,[COYS],662,jack grealish playing man city like,[COYS],2021-08-15 17:30:58,0.0,Primo,1,"{'neg': 0.0, 'neu': 0.482, 'pos': 0.518, 'comp...",0.5106,Positive,Tottenham
4,1257180002,tax_i_,[COYS],449946,come spurs,[COYS],2021-08-15 17:31:00,0.0,Primo,1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,Neutral,Tottenham
5,24518986,littlejason,[COYS],701,boys,[COYS],2021-08-15 17:31:00,0.0,Primo,1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,Neutral,Tottenham
6,338670340,rkc3indy,"[EPL, THFC, COYS, FuckManShitty]",454,matchday buffalo wild wings,"[COYS, EPL, FuckManShitty, THFC]",2021-08-15 17:31:00,0.0,Primo,1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,Neutral,Tottenham
7,1287748779324641290,Bez_FPL,[COYS],662,come,[COYS],2021-08-15 17:31:02,0.0,Primo,1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,Neutral,Tottenham
8,1311955295686492163,HelenUpsher,[Tottenham],339,watching tottenham v man c,[Tottenham],2021-08-15 17:31:02,0.0,Primo,1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,Neutral,Tottenham
9,1385194605365321729,DPT_34,"[COYS, THFC]",94,levys wife,"[COYS, THFC]",2021-08-15 17:31:03,0.0,Primo,1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,Neutral,Tottenham


In [ ]:
Tott = df_T_clean[df_T_clean['tifo'] == "Tottenham"]
len(Tott)

12644

In [ ]:
City = df_T_clean[df_T_clean['tifo'] == "Manchester City"]
len(City)

5009

In [ ]:
Neut = df_T_clean[df_T_clean['tifo'] == "Neutral"]
len(Neut)

30745

Infine, si binarizzano le label sul sentiment per facilitare il processing successivo.

In [ ]:
def Neut_bin(label):
  if label == "Neutral":
    return 1
  else:
    return 0

def Pos_bin(label):
  if label == "Positive":
    return 1
  else:
    return 0

def Neg_bin(label):
  if label == "Negative":
    return 1
  else:
    return 0

In [ ]:
df_T_clean["Positive"] = df_T_clean["comp_score"].apply(Pos_bin)
df_T_clean["Negative"] = df_T_clean["comp_score"].apply(Neg_bin)
df_T_clean["Neutral"] = df_T_clean["comp_score"].apply(Neut_bin)

df_T_clean.head(15)

,user_id,screen_name,hashtags,followers,text_clean,compl_hashtags,time,minute_from_start,Moment,Min_Cronaca,scores,compound,comp_score,tifo,Positive,Negative,Neutral
0,576167561,untriplesix,[COYS],326,come boys,[COYS],2021-08-15 17:30:53,0.0,Primo,1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,Neutral,Tottenham,0,0,1
1,266251364,lazqetjohnson,[COYS],24053,fans best england country mile,[COYS],2021-08-15 17:30:53,0.0,Primo,1,"{'neg': 0.0, 'neu': 0.488, 'pos': 0.512, 'comp...",0.6369,Positive,Tottenham,1,0,0
2,276233686,RyanDHobart,[coys],635,know woman straight ignored daniel levy shes n...,[coys],2021-08-15 17:30:58,0.0,Primo,1,"{'neg': 0.331, 'neu': 0.508, 'pos': 0.161, 'co...",-0.2500,Negative,Neutral,0,1,0
3,161413417,colinblouin,[COYS],662,jack grealish playing man city like,[COYS],2021-08-15 17:30:58,0.0,Primo,1,"{'neg': 0.0, 'neu': 0.482, 'pos': 0.518, 'comp...",0.5106,Positive,Tottenham,1,0,0
4,1257180002,tax_i_,[COYS],449946,come spurs,[COYS],2021-08-15 17:31:00,0.0,Primo,1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,Neutral,Tottenham,0,0,1
5,24518986,littlejason,[COYS],701,boys,[COYS],2021-08-15 17:31:00,0.0,Primo,1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,Neutral,Tottenham,0,0,1
6,338670340,rkc3indy,"[EPL, THFC, COYS, FuckManShitty]",454,matchday buffalo wild wings,"[COYS, EPL, FuckManShitty, THFC]",2021-08-15 17:31:00,0.0,Primo,1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,Neutral,Tottenham,0,0,1
7,1287748779324641290,Bez_FPL,[COYS],662,come,[COYS],2021-08-15 17:31:02,0.0,Primo,1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,Neutral,Tottenham,0,0,1
8,1311955295686492163,HelenUpsher,[Tottenham],339,watching tottenham v man c,[Tottenham],2021-08-15 17:31:02,0.0,Primo,1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,Neutral,Tottenham,0,0,1
9,1385194605365321729,DPT_34,"[COYS, THFC]",94,levys wife,"[COYS, THFC]",2021-08-15 17:31:03,0.0,Primo,1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,Neutral,Tottenham,0,0,1


Si conservano solamente le colonne d'interesse

In [ ]:
df_final = df_T_clean.drop('scores', axis = 1)
df_final.head(15)

,user_id,screen_name,hashtags,followers,text_clean,compl_hashtags,time,minute_from_start,Moment,Min_Cronaca,compound,comp_score,tifo,Positive,Negative,Neutral
0,576167561,untriplesix,[COYS],326,come boys,[COYS],2021-08-15 17:30:53,0.0,Primo,1,0.0000,Neutral,Tottenham,0,0,1
1,266251364,lazqetjohnson,[COYS],24053,fans best england country mile,[COYS],2021-08-15 17:30:53,0.0,Primo,1,0.6369,Positive,Tottenham,1,0,0
2,276233686,RyanDHobart,[coys],635,know woman straight ignored daniel levy shes n...,[coys],2021-08-15 17:30:58,0.0,Primo,1,-0.2500,Negative,Neutral,0,1,0
3,161413417,colinblouin,[COYS],662,jack grealish playing man city like,[COYS],2021-08-15 17:30:58,0.0,Primo,1,0.5106,Positive,Tottenham,1,0,0
4,1257180002,tax_i_,[COYS],449946,come spurs,[COYS],2021-08-15 17:31:00,0.0,Primo,1,0.0000,Neutral,Tottenham,0,0,1
5,24518986,littlejason,[COYS],701,boys,[COYS],2021-08-15 17:31:00,0.0,Primo,1,0.0000,Neutral,Tottenham,0,0,1
6,338670340,rkc3indy,"[EPL, THFC, COYS, FuckManShitty]",454,matchday buffalo wild wings,"[COYS, EPL, FuckManShitty, THFC]",2021-08-15 17:31:00,0.0,Primo,1,0.0000,Neutral,Tottenham,0,0,1
7,1287748779324641290,Bez_FPL,[COYS],662,come,[COYS],2021-08-15 17:31:02,0.0,Primo,1,0.0000,Neutral,Tottenham,0,0,1
8,1311955295686492163,HelenUpsher,[Tottenham],339,watching tottenham v man c,[Tottenham],2021-08-15 17:31:02,0.0,Primo,1,0.0000,Neutral,Tottenham,0,0,1
9,1385194605365321729,DPT_34,"[COYS, THFC]",94,levys wife,"[COYS, THFC]",2021-08-15 17:31:03,0.0,Primo,1,0.0000,Neutral,Tottenham,0,0,1


In [ ]:
df_final.to_csv("Match_clean.csv")